ETL Steam Games

In [2]:
#Importando librerias
import json
import numpy as np
import pandas as pd
import ast

1. Extraccion de datos

In [5]:
steam_1 = pd.read_json('data/steam_games.json.gz',lines=True,compression='gzip')
steam_1.head(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


2. Exploración de datos

In [7]:
#reviso si tiene algun dato
steam_1.tail(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"
120444,None,None,Maze Run VR,None,http://store.steampowered.com/app/681550/Maze_...,None,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,1.0,681550.0,None


2.1 Busco valores nulos

In [8]:
steam_1.isna().sum() #Buscamos y sumamos los valores nulos por cada categoria.

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

In [9]:
#Porcentajes de ratios de nulos
na_ratio = ((steam_1.isnull().sum() / len(steam_1))*100).sort_values(ascending = False)
print(na_ratio)

publisher       80.004982
developer       76.058782
genres          76.045498
release_date    75.035909
title           75.021794
price           74.463033
specs           73.876043
tags            73.455104
app_name        73.321433
reviews_url     73.321433
id              73.321433
url             73.319773
early_access    73.319773
dtype: float64


In [11]:
#veo cuantos registros se eliminaran
steam_1.shape

(120445, 13)

In [12]:
#Elimino todos los nulos
steam_2 = steam_1.dropna(how="all").reset_index(drop=True)
steam_2.shape

(32135, 13)

2.2 Busco valores repetidos

In [44]:
steam_3=steam_2.drop_duplicates(subset="id")
steam_3.shape

(32133, 13)

2.3 Algunas variables parecen ser listas anidadas, esto es un formato usual cuando se guarda en formato jason,
    por lo cual se revisaran cuantas variables se encuentran asi y desanidarlas en el caso que se necesite usarlas.

In [45]:
tipo_data = {"columna":[],"tipos_de_datos":[]} #genero un diccionario vacio para ir almacenando lo que genere el bucle

for columna in steam_3.columns: #un bucle que va recorriendo 
    tipo_data["columna"].append(columna)
    tipo_data["tipos_de_datos"].append(steam_3[columna].apply(type).unique())

analisis= pd.DataFrame(tipo_data)
analisis

,columna,tipos_de_datos
0,publisher,"[<class 'str'>, <class 'NoneType'>]"
1,genres,"[<class 'list'>, <class 'NoneType'>]"
2,app_name,"[<class 'str'>, <class 'NoneType'>]"
3,title,"[<class 'str'>, <class 'NoneType'>]"
4,url,[<class 'str'>]
5,release_date,"[<class 'str'>, <class 'NoneType'>]"
6,tags,"[<class 'list'>, <class 'NoneType'>]"
7,reviews_url,"[<class 'str'>, <class 'NoneType'>]"
8,specs,"[<class 'list'>, <class 'NoneType'>]"
9,price,"[<class 'float'>, <class 'str'>, <class 'NoneT..."


Todos aquellos que tengan 'list', significa que hay una lista anidada dentro de esa variable

Para crear las funciones solamente requerimos de 4 variables de la tabla steam, pero luego para calcular el modelo de recomedanción se requiere realizar un EDA para verificar cuales serian funcionales al modelo. Por lo cual en una primera instancia transformaremos algunas variables y luego crearemos dos tablas, una exclusivamente para las 4 funciones y otra para el modelo de recomendación.
Esto se realiza debido al consumo de memoria de la API es limitada, lo cual para ser más eficiente, se requiere las bases de datos exclusivamente las necesarias.

2.4 Creación de año

In [46]:
steam_3['Año de lanzamiento'] = steam_3['release_date'].str.extract(r'(\d{4})').fillna('0')
steam_3.head(3)

C:\Users\molin\AppData\Local\Temp/ipykernel_18600/1470025964.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_3['Año de lanzamiento'] = steam_3['release_date'].str.extract(r'(\d{4})').fillna('0')


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,Año de lanzamiento
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro,2018
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL,2018
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,Poolians.com,2017


2.5 Transformación de id, para poder realizar cruces

In [40]:
#Para convertir ID, debo asegurarme que no haya NaN
print(steam_3[steam_3['id'].isnull()])

   publisher genres app_name title                             url  \
74      None   None     None  None  http://store.steampowered.com/   

   release_date  tags reviews_url specs  price  early_access  id developer  \
74         None  None        None  None  19.99           0.0 NaN      None   

   Año de lanzamiento  
74                  0  


In [48]:
steam_4 = steam_3.dropna(subset=['id'])
steam_4.shape

(32132, 14)

In [49]:
#transformo y renombro
steam_4['id']=steam_4['id'].apply(np.int64)
steam_4= steam_4.rename(columns={"id":"item_id"})

C:\Users\molin\AppData\Local\Temp/ipykernel_18600/1581868648.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_4['id']=steam_4['id'].apply(np.int64)


In [50]:
#verifico
steam_4.dtypes

publisher              object
genres                 object
app_name               object
title                  object
url                    object
release_date           object
tags                   object
reviews_url            object
specs                  object
price                  object
early_access          float64
item_id                 int64
developer              object
Año de lanzamiento     object
dtype: object

2.6 Transformar la variable genero

Se crean las filas necesarias por la cantidad de generos registrados por cada id, con lo cual si un id tiene 5 generos, ahora habrá 5 ids repetidos pero con generos unicos.

In [64]:
steam_5 = steam_4.explode("genres")
steam_5.shape

(74833, 14)

In [65]:
steam_5.head(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,item_id,developer,Año de lanzamiento
0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140,Kotoshiro,2018
0,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140,Kotoshiro,2018
0,Kotoshiro,Indie,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140,Kotoshiro,2018


2.7 Me quedo con las variables que son necesarias

In [72]:
 steam_6= steam_5[['item_id','app_name','title','genres','Año de lanzamiento','publisher','developer','specs','price']].copy()
steam_6.head(2) #queda como la tabla final para el sistema de recomendación

,item_id,app_name,title,genres,Año de lanzamiento,publisher,developer,specs,price
0,761140,Lost Summoner Kitty,Lost Summoner Kitty,Action,2018,Kotoshiro,Kotoshiro,[Single-player],4.99
0,761140,Lost Summoner Kitty,Lost Summoner Kitty,Casual,2018,Kotoshiro,Kotoshiro,[Single-player],4.99


In [73]:
steam_7 =  steam_6[['item_id','app_name','title','genres','Año de lanzamiento']].copy()
steam_7.head(2) #queda como la tabla final para las funciones

,item_id,app_name,title,genres,Año de lanzamiento
0,761140,Lost Summoner Kitty,Lost Summoner Kitty,Action,2018
0,761140,Lost Summoner Kitty,Lost Summoner Kitty,Casual,2018


2.8 guardo las tablas para reutilizarlas más adelante

In [74]:
steam_7_csv = "data/steam_7_api.csv"
steam_7.to_csv(steam_7_csv , index=False, encoding="utf-8")

In [75]:
steam_6_csv = "data/steam_6_modelo.csv"
steam_6.to_csv(steam_6_csv , index=False, encoding="utf-8")

*****************************************************